In [ ]:
import os
import pylab
import string
import fdsreader
import matplotlib
import subprocess

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from importlib import reload
from matplotlib.colors import LinearSegmentedColormap
from scipy.ndimage import gaussian_filter
import spotpy
from scipy.stats.qmc import LatinHypercube
from pyDOE import lhs
import random
import shutil
import io

# LatinHypercube

## Explain

Latin Hypercube Sampling (LHS) is a statistical method used for generating a representative sample of parameter values from a multi-dimensional parameter space. The method involves dividing each dimension of the parameter space into equally-sized intervals and selecting one random point from each interval, such that each point is chosen exactly once. The resulting sample is thus representative of the parameter space in a more even and efficient way than a completely random sampling approach

generate the LHS sample using `LatinHypercube` from `scipy.stats.qmc`:

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats.qmc import LatinHypercube

# Define the number of dimensions and the number of samples to generate
n_dims = 2
n_samples = 10

# Define the parameter ranges for each dimension
params_ranges = np.array([[0, 1], [0, 1]])

# Generate a Latin Hypercube sample within the parameter ranges
sample = LatinHypercube(d=n_dims, seed=123).random(n_samples) * (params_ranges[:, 1] - params_ranges[:, 0]) + params_ranges[:, 0]

# Create a grid to show the parameter space
xx, yy = np.meshgrid(np.linspace(params_ranges[0, 0], params_ranges[0, 1], 10),
                     np.linspace(params_ranges[1, 0], params_ranges[1, 1], 10))
grid = np.column_stack([xx.ravel(), yy.ravel()])

# Plot the resulting sample and the parameter space grid
fig, axs = plt.subplots(ncols=2, figsize=(10, 4))
axs[0].scatter(sample[:, 0], sample[:, 1])
axs[0].set_xlabel('Parameter 1')
axs[0].set_ylabel('Parameter 2')
axs[0].set_title('Latin Hypercube Sample')

axs[1].scatter(grid[:, 0], grid[:, 1], color='gray')
axs[1].scatter(sample[:, 0], sample[:, 1])
axs[1].set_xlabel('Parameter 1')
axs[1].set_ylabel('Parameter 2')
axs[1].set_title('Parameter Space')
plt.show()


Latin Hypercube Sampling is designed to spread the sample points evenly throughout the parameter space, while still maintaining randomness. In other words, the LHS algorithm aims to fill the parameter space with points in such a way that each point is representative of a different region of the space, while avoiding clusters or gaps of points.



## Case Variations

Define the parameter settings:

* PATH_LENGTH - constant just min / max <br>
* ANGLE_INCREMENT - constant just min / max  <br>
* NUMBER_RADIATION_ANGLES - min TO 1000 <br>
* +- 20% HRRPUA <br>
* +- 20% SOOT_YIELD=0.024 Table A.39 - SFPE page 3466 <br>
* +- 20% radiative fraction PROPANE=0.3 Table 16.1 - FUG page 190 <br>
* +- 10 TMPA Domain -  default =20 <br>
* +- 0.8 - 0.99 EMISSIVITY Burner <br>
* +- 0.8 - 0.99 EMISSIVITY Panel <br>

In [ ]:
#new min/max
HRRPUA=333
print(f'HRRPUA -20% = {round(HRRPUA*0.8,2)}')
print(f'HRRPUA +20% = {round(HRRPUA*1.2,2)}')
SY= 0.024 
print(f'SY -20% = {round(SY*0.8,5)}')
print(f'SY +20% = {round(SY*1.2,5)}')
rf= 0.3 
print(f'rf -20% = {round(rf*0.8,2)}')
print(f'rf +20% = {round(rf*1.2,2)}')

###
#path_length max 
point1 = np.array([-0.15, -0.3, 1.2])
point2 = np.array([0.15, 0.3, 1.2])

#diagonal corner to corner of each panel 
distance = np.linalg.norm(point1 - point2)
print("Distance between corner to the corner of the panels (diagonal) for max path_length:", distance)

### Parameter Informations for the script
Propeties, Ranges and Sampling Method for the next step
* params_info: **[("Name (here: FDS Command)", min Value, max Value, "sampling methods")]**
* sampling methods: 
    * 'simple' just switching randomly between min and max value
    * 'LHS' using Latin Hypercube Sampling in the given range of min and max value

In [ ]:
# EMISSIVITY_Burner and EMISSIVITY_Panel are placeholder for unique labeling in the FDS Template 
params_info = [
                ('PATH_LENGTH', 0.1, 1, 'simple'),
                ('ANGLE_INCREMENT', 1, 5, 'simple'),
                ('NUMBER_RADIATION_ANGLES', 100, 1000, 'LHS'),
                ('HRRPUA', 266.4, 399.6, 'LHS'),
                ('SOOT_YIELD', 0.0192, 0.0288, 'LHS'),
                ('RADIATIVE_FRACTION', 0.24, 0.36, 'LHS'),
                ('TMPA', 10, 25, 'LHS'),
                ('EMISSIVITY_Burner', 0.8, 0.99, 'LHS'),
                ('EMISSIVITY_Panel', 0.8, 0.99, 'LHS')
              ]

In [ ]:
# Function to write data to CSV file similar to cone script 
def write_csv(filename, data):
    with open(filename, 'w', newline='') as csvfile:
        csvwriter = csv.writer(csvfile)
        for row in data:
            csvwriter.writerow(row)

# Function to create a directory for better handling
def create_directory(path):
    if not os.path.exists(path):
        os.makedirs(path)

#####IMPORTANT ADJUSTMENTS OF CASES AND SAMPLES#######
# Defining propeties
n_dims = len(params_info) # for n parameters
n_samples = 2 # for n sample sets in each cases
cases = 2 # n cases (generations of the seed - with each case the seed value is new randomly implemented)

# Defining the parameters, ranges, and sampling methods - PATH_LENGTH distance digonal max 0.67082
params_info = params_info

# Split names, ranges, and sampling methods
params_names = np.array([info[0] for info in params_info])
params_ranges = np.array([[info[1], info[2]] for info in params_info])
sampling_methods = np.array([info[3] for info in params_info])

# Defining the directory where the cases will be located
fds_input_dir = 'P:/Shared/Studium/Masterthesis/ParallelPanelBurnerSetup/RunReports/Cases_Launch'

# FDS template location
fds_input_temp = f'{fds_input_dir[0:-13]}/Templates/fds_input_template.fds'
# Defining the name of the FDS file
fds_label = "MaCFP_Burner_"

dfs = [] 

# Looping over n cases
for i in range(cases):
    # Generate a random range for the integer
    start = random.randint(0, 10000)
    stop = random.randint(start+1, 10000)

    # Generate a random integer for the seed within the generated range
    random_integer = random.randrange(start, stop)
    
    # Creating a directory for the current cases and name it 
    cases = os.path.join(fds_input_dir, f"case{i}_{random_integer}")
    create_directory(cases)
    
    # Generating a sample using Latin Hypercube Sampling or simple min-max sampling (random pick)
    samples = []
    for (min_val, max_val), sampling_method in zip(params_ranges, sampling_methods):

        if sampling_method == 'LHS':
            sample = LatinHypercube(d=1, seed=random_integer).random(n_samples) * (max_val - min_val) + min_val
        elif sampling_method == 'simple':
            sample = np.random.choice([min_val, max_val], size=n_samples)

        samples.append(sample)

    sample = np.column_stack(samples).astype(float)
    
    # Convert the array to a DataFrame
    df = pd.DataFrame(sample, columns=[p[0] for p in params_info])
    dfs.append(df)  

    # Save the parameter values to a CSV file 
    df.to_csv(f'{fds_input_dir}/../TableData/case{i}_param_values.csv', index=False)
  
    # Looping over each parameter set for n samples
    for j, param_set in enumerate(sample):
        # Creating a directory for the current sample within the cases directory
        sample_dir = os.path.join(cases, f"{fds_label[0:6]}CASE{i}_sample{j}")
        create_directory(sample_dir)
               
        ####FDS file
        # Reading the contents of the initial FDS file
        with open(os.path.join(fds_input_dir, fds_input_temp), 'r') as fds_input:
            fds_input_content = fds_input.readlines()
        
        # Add the CHID with the current sample and cases to the &HEAD line
        for idx, line in enumerate(fds_input_content):
            if "&HEAD" in line:
                fds_input_content[idx] = f'&HEAD CHID="{fds_label}CASE{i}_sample{j}", {line[29:]}\n'
                break
        
        # Modifying the FDS file with the current parameter set
        # using Placeholder to EMISSIVITY_Panel / EMISSIVITY_Burner for unique values
        for k, (param, _, _, _) in enumerate(params_info):
            for idx, line in enumerate(fds_input_content):
                if "EMISSIVITY_Panel" in line:
                    fds_input_content[idx] = f"      EMISSIVITY = {param_set[k]}\n"
                elif "EMISSIVITY_Burner" in line:
                    fds_input_content[idx] = f"      EMISSIVITY = {param_set[k]}\n"
                elif param in line:
                    fds_input_content[idx] = f"      {param} = {param_set[k]}\n"
                    break
                    
        ####pleiades file 
        #pleiades file location
        pleiades_file = f'{fds_input_dir[0:-13]}/Templates/job_fds.pleiades'

        # Open and read the pleiades file
        with open(pleiades_file, 'r') as input_file:
            file_content = input_file.read()

        # Replace '******' with the appropriate label
        new_content = file_content.replace('******', f'{fds_label}CASE{i}_sample{j}')
        
        fds_input_label_gen_sample = f"{fds_label}CASE{i}_sample{j}.fds"
        fds_input_filepath = os.path.join(sample_dir, fds_input_label_gen_sample)
        
        # Save the result with Unix line endings
        out_file_path = f'{sample_dir}/job_fds.pleiades'
        with io.open(out_file_path, 'w', newline='\n') as output_file:
            output_file.write(new_content)
        
        # Saving the modified FDS file in the folder of the sample 
        with open(fds_input_filepath, 'w') as new_fds_file:
            new_fds_file.writelines(fds_input_content)

# Concatenate all DataFrames into a single DataFrame
param_df = pd.concat(dfs, axis=0, ignore_index=True)
print("DONE!")

In [ ]:
pd.read_csv(f'{fds_input_dir}/../TableData/case0_param_values.csv', header=0)